In [ ]:
!pip install -r learning_to_simulate/requirements.txt

In [ ]:
!bash ./learning_to_simulate/download_dataset.sh WaterDropSample /tmp/datasets

In [ ]:
!mkdir -p /tmp/models
!python -m learning_to_simulate.train \
    --data_path=/tmp/datasets/WaterDropSample \
    --model_path=/tmp/models/WaterDropSample \
    --num_steps=1000

In [ ]:
!mkdir -p /tmp/rollouts
!python -m learning_to_simulate.train \
    --mode="eval_rollout" \
    --data_path=/tmp/datasets/WaterDropSample \
    --model_path=/tmp/models/WaterDropSample \
    --output_path=/tmp/rollouts/WaterDropSample \

In [ ]:
%matplotlib inline
import pickle
from matplotlib import animation
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

TYPE_TO_COLOR = {
    3: "black",  # Boundary particles.
    0: "green",  # Rigid solids.
    7: "magenta",  # Goop.
    6: "gold",  # Sand.
    5: "blue",  # Water.
}


def main():

    with open('./rollout_test_0.pkl', "rb") as file:
        rollout_data = pickle.load(file)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    plot_info = []
    for ax_i, (label, rollout_field) in enumerate(
      [("Ground truth", "ground_truth_rollout"),
       ("Prediction", "predicted_rollout")]):
        # Append the initial positions to get the full trajectory.
        trajectory = np.concatenate([
            rollout_data["initial_positions"],
            rollout_data[rollout_field]], axis=0)
        ax = axes[ax_i]
        ax.set_title(label)
        bounds = rollout_data["metadata"]["bounds"]
        ax.set_xlim(bounds[0][0], bounds[0][1])
        ax.set_ylim(bounds[1][0], bounds[1][1])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect(1.)
        points = {
            particle_type: ax.plot([], [], "o", ms=2, color=color)[0]
            for particle_type, color in TYPE_TO_COLOR.items()}
        plot_info.append((ax, trajectory, points))

    num_steps = trajectory.shape[0]

    def update(step_i):
        outputs = []
        for _, trajectory, points in plot_info:
            for particle_type, line in points.items():
                mask = rollout_data["particle_types"] == particle_type
                line.set_data(trajectory[step_i, mask, 0],
                              trajectory[step_i, mask, 1])
                outputs.append(line)
        return outputs

    unused_animation = animation.FuncAnimation(
      fig, update,
      frames=np.arange(0, num_steps, 3), interval=10)
    return unused_animation



anim = main()
HTML(anim.to_jshtml())